In [1]:
!pip install html5lib
!pip install tqdm
!pip install pandas
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import html5lib

In [31]:
url = "https://books.toscrape.com"
response = requests.get(url)
soup = BeautifulSoup(response.text)

all_current = soup.find_all("li", {"class": "current"})
current = all_current[0]
encoded = current.encode_contents().decode("utf-8")
splits = encoded.split("\n")
page_string = splits[2].strip()
splits = page_string.split("of")
page_count_string = splits[1].strip()
page_count = int(page_count_string)

page_min = page_count - 9
page_max = page_count + 1

link_list = []
for i in range(page_min, page_max):
    page = f'https://books.toscrape.com/catalogue/page-{i}.html'
    response = requests.get(page)
    soup = BeautifulSoup(response.text)
    
    for i in soup.findAll("h3"):
        for link in i.find_all('a',href = True):
            part_link = link["href"]                                  
            full_link = f'https://books.toscrape.com/catalogue/{part_link}'
            link_list.append(full_link)

print(len(link_list))

200


In [100]:
dictionary_list = []
counter = 0
length = len(link_list)
for page in link_list:
    counter += 1;
    print(f'{counter}/{length}') # drucke den fortschritt aus, damit man sieht das was passiert
    response = requests.get(page)
    soup = BeautifulSoup(response.text)

    header = soup.find_all("h1")[0] # hole den ersten (einzigsten) header
    name = header.encode_contents().decode("utf-8") # erstelle einen string aus dem header

    is_first = True
    category_link = ""
    for a in soup.find_all('a', href=True): # finde alle a
        href = a['href']
        if "category" in href: # finde alle category links (2)
            if is_first:
                is_first = False # wenn der erste link gefunden wurde, setze is_second auf False
            else:
                category_link = href # zweiter category link gefunden!
                break # wir können die loop beenden, weil wir den link gefunden haben

    splits = category_link.split("/")
    category = splits[3].split("_")[0] # hole die category aus dem link

    price = 0;
    availability = 0;

    keys = soup.find_all("th"); # hole alle produkt informations keys
    values = soup.find_all("td"); # hole alle produkt informations values
    zipped = zip(keys, values); # füge keys und values zusammen

    for key, value in zipped: # iteriere zipped
        key_string = key.encode_contents().decode("utf-8") # erstelle einen string aus dem html element
        value_string = value.encode_contents().decode("utf-8") # erstelle einen string aus dem html element

        # print(f'{key_string} -> {value_string}')

        if "Price (incl. tax)" in key_string: # wenn price, dann hole die float aus dem string
            price_string = value_string[2:]
            price = float(price_string)
        elif "Availability" in key_string: # wenn availability, hole den int aus dem string
            splits = value_string.split("(")
            splits = splits[1].split(" ")
            availabilty_string = splits[0]
            availability = int(availabilty_string)

    # generiere dictionary
    dictionary = {
        "name": name,
        "category": category,
        "price": price,
        "availability": availability,
    }
    
    dictionary_list.append(dictionary)

# erstelle dataframe
data_frame = pd.DataFrame(data=dictionary_list)
print(data_frame)

1/200
2/200
3/200
4/200
5/200
6/200
7/200
8/200
9/200
10/200
11/200
12/200
13/200
14/200
15/200
16/200
17/200
18/200
19/200
20/200
21/200
22/200
23/200
24/200
25/200
26/200
27/200
28/200
29/200
30/200
31/200
32/200
33/200
34/200
35/200
36/200
37/200
38/200
39/200
40/200
41/200
42/200
43/200
44/200
45/200
46/200
47/200
48/200
49/200
50/200
51/200
52/200
53/200
54/200
55/200
56/200
57/200
58/200
59/200
60/200
61/200
62/200
63/200
64/200
65/200
66/200
67/200
68/200
69/200
70/200
71/200
72/200
73/200
74/200
75/200
76/200
77/200
78/200
79/200
80/200
81/200
82/200
83/200
84/200
85/200
86/200
87/200
88/200
89/200
90/200
91/200
92/200
93/200
94/200
95/200
96/200
97/200
98/200
99/200
100/200
101/200
102/200
103/200
104/200
105/200
106/200
107/200
108/200
109/200
110/200
111/200
112/200
113/200
114/200
115/200
116/200
117/200
118/200
119/200
120/200
121/200
122/200
123/200
124/200
125/200
126/200
127/200
128/200
129/200
130/200
131/200
132/200
133/200
134/200
135/200
136/200
137/200
138/200
139/

In [101]:
print("excercise 2")

excercise 2


In [147]:
# get only 1 laureate, so that we can access meta.count, to know the total amount of laureates
initial_url = "https://masterdataapi.nobelprize.org/2.1/laureates?offset=0&limit=1"
response = requests.get(initial_url)
data = response.json()
meta = data['meta']
total_laureates_count = meta['count']

print(f'found {total_laureates_count} laureates')

# get ALL laureates
url = f'https://masterdataapi.nobelprize.org/2.1/laureates?sort=asc&offset=0&limit={total_laureates_count}'
response = requests.get(url);
data = response.json()
all_laureates = data['laureates']

print(len(all_laureates))

found 981 laureates
fetching... https://masterdataapi.nobelprize.org/2.1/laureates?sort=asc&offset=0&limit=981
parsing to json...
981


In [150]:
laureates = []

for laureate in all_laureates:
    if "fullName" in laureate: # if laureate doesn't contain "fullName", it is not a person
        name = laureate['fullName']['en']
        birthday = laureate['birth']['date']
        nobel_prizes = []
        
        for nobel_prize in laureate['nobelPrizes']:
            dateAwarded = ""
            if 'dateAwarded' in nobel_prize: # if 'dateAwarded' doesn't exist, use 'awardYear'
                dateAwarded = nobel_prize['dateAwarded']
            else:
                dateAwarded = nobel_prize['awardYear']
            
            dictionary = {
                'dateAwarded': dateAwarded,
                'category': nobel_prize['category']['en'],
            }
            
            nobel_prizes.append(dictionary)
        
        birth_country = "unknown"
        if 'place' in laureate['birth']: # if birthplace exists, take it, otherwise it will be "unkown"
            birth_country = laureate['birth']['place']['country']['en']
        
        wikipedia = laureate['wikipedia']['english']
        
        # create dictionary and add to list
        dictionary = {
            'name': name,
            'birthday': birthday,
            'nobel_prizes': nobel_prizes,
            'birth_country': birth_country,
            'wikipedia': wikipedia
        }
        
        laureates.append(dictionary)

print(len(laureates))

954
